### Soft Dedup 전 N-gram Dataframe 생성

In [1]:
import pickle

## Load pickle
with open("/opt/output/perm/analysis/ngram_result.pkl","rb") as f:
    ngram_result = pickle.load(f)

In [2]:
import pandas as pd
# 초기화
ngramdf = pd.DataFrame(columns=["N", "Ngram_syscall", "num_of_exploit"])
item = dict()
for i in range(len(ngram_result)):
    for ngram_syscall, cnt in ngram_result[i].items():
        item["N"] = i+2
        item["Ngram_syscall"] = ngram_syscall.split(",")
        item["num_of_exploit"] = cnt
        ngramdf = ngramdf.append(item, ignore_index=True)
        item = dict()
ngramdf_origin = ngramdf.copy()
ngramdf_origin

,N,Ngram_syscall,num_of_exploit
0,2,"[12, 41]",3
1,2,"[257, 5]",40
2,2,"[107, 117]",3
3,2,"[3, 103]",3
4,2,"[8, 5]",4
...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2


In [3]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find /opt/output/perm/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("/opt/output/perm/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"/opt/output/perm/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)

In [4]:
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)

In [5]:
import numpy as np
def find_EID_with_ngram(ngram, ngram_cnt):
    EID_with_ngram  = list()
    for EID, pathset in eid_all_path_dict.items():
        if ngram_cnt == len(EID_with_ngram):
            return EID_with_ngram
        for path in pathset:
            strpath = "," + ",".join(path) + ","
            if ngram in strpath:
                EID_with_ngram.append(EID)
                break
    if not EID_with_ngram: # if no EID
        return np.nan
    else:
        return EID_with_ngram

In [6]:
# ngrma이 나오는 EID 저장
import numpy as np
ngramdf["EIDs"] = np.nan
for index, row in ngramdf.iterrows():
    strngram = "," + ",".join(row.Ngram_syscall) + ","
    ngramdf["EIDs"][index] = find_EID_with_ngram(strngram, row.num_of_exploit)
ngramdf

/tmp/ipykernel_2896765/725562346.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngramdf["EIDs"][index] = find_EID_with_ngram(strngram, row.num_of_exploit)
/home/mini/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2,NaN
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,NaN
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN


In [7]:
# N-gram 처리 (시스템콜 아닌 것들 바꿔주기, 유저함수 삭제)
import re

for index, row in ngramdf.iterrows():
    if "setns" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("setns","308").split(",")
    if "time" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("time","201").split(",")
    if "capset" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("capset","126").split(",")
    if "fstat" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("fstat","5").split(",")
    if "uname" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("uname","63").split(",")
    if "__NR_vm86old" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("__NR_vm86old","436").split(",")
    if "__NR_socketcall" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("__NR_socketcall","437").split(",")
    if "sigaction" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("sigaction","13").split(",")
    if "ioctl" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("ioctl","16").split(",")
    if "kmalloc" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("kmalloc","12").split(",")
    if "shutdown" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("shutdown","48").split(",")
    if "msgrcv" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("msgrcv","70").split(",")
    ngramdf.iloc[index]['Ngram_syscall'] = re.findall(r'\d+', ",".join(row.Ngram_syscall)).copy()
ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2,NaN
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,NaN
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN


In [8]:
import numpy as np
# Ngram 처리 후 변경 사항 처리
for index, row in ngramdf.iterrows():
    # Ngram_syscall 없는 애들은 NaN로 바꿔주기
    if not row.Ngram_syscall:
        ngramdf.iloc[index]['N'] = np.nan
    elif not isinstance(row.Ngram_syscall, list):
        ngramdf.iloc[index]['N'] = np.nan
    elif len(row.Ngram_syscall) == row.N:      
        ngramdf.iloc[index]['N'] = len(row.Ngram_syscall)

ngramdf = ngramdf.dropna(axis=0)
ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11329,59,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
11332,59,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
11335,60,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
11336,60,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"


In [9]:
# pickle로 저장
import pickle

with open("defore_soft_dedup_ngramdf.pkl","wb") as f:
    pickle.dump(ngramdf, f)

In [22]:
# pickle로 저장
import pickle

with open("defore_soft_dedup_ngramdf.pkl","rb") as f:
    ngramdf = pickle.load(f)

In [24]:
import subprocess
# 시스템콜 번호를 시스템콜 이름으로 바꾸기 위함 함수
def convert_num_name(sysnum):
    if sysnum == 436:
        return "vm86old"
    if sysnum == 437:
        return "socketcall"
    if sysnum == "clone":
        return "clone"
    if sysnum == "write":
        return "write"
    cmd = f'grep " {sysnum}$" /tmp/syscall_64.txt'
    sys_name = subprocess.check_output(cmd, shell=True).decode().strip().split()[1].replace("__NR_","")
    return sys_name

In [25]:
import subprocess
# 시스템콜 이름을 시스템콜 번호로 바꾸기 위함 함수
def convert_name_num(sysname):
    if sysname == "vm86old":
        return 436
    if sysname == "socketcall":
        return 437
    cmd = f'grep "\s{sysname}\s" /tmp/syscall_64.txt'
    try:
        sys_num = subprocess.check_output(cmd, shell=True).decode().strip().split()[0]
    except:
        cmd = f'grep "\s{sysname}" /tmp/syscall_64.txt'
        sys_num = subprocess.check_output(cmd, shell=True).decode().strip().split()[0]
    return sys_num

# Confine과 비교

### Confine Seccomp Profile 읽기

In [13]:
# Confine Git Repository Clone
import os
# os.system("git clone https://github.com/shamedgh/confine.git")

In [26]:
# Confine Seccomp profile 파일 명 및 파일 들고오기
import subprocess
cmd = "find confine/sample.results/ -name *.json"
json_files = subprocess.check_output(cmd, shell=True).decode().strip().split()
pwd = subprocess.check_output("pwd", shell=True).decode().strip()

# Json -> Dataframe
import pandas as pd
import json
from pandas import json_normalize

confine_profile = dict() # key: profile name, value: dataframe
for file in json_files:
    loc = pwd + "/" + file
    with open(loc, 'r', encoding='utf-8') as f:
        # value
        profile = json.load(f)
        nomalized_json = json_normalize(profile['syscalls'])
        value = nomalized_json.drop(['action','args'],axis='columns')
        # key
        remove_sentence = pwd+"/confine/sample.results/"
        key = loc.replace(remove_sentence,"").replace(".seccomp.json","")
        confine_profile[key] = value
        
len(confine_profile)

148

In [27]:
# Example
confine_profile["httpd"]

,name
0,pwrite64
1,readv
2,msync
3,mincore
4,pause
...,...
165,pwritev2
166,pkey_mprotect
167,pkey_alloc
168,pkey_free


In [28]:
# Dataframe 하나로 합치기
# Columns = {Applicaton Name, Seccomp Syscall, EID}
import pandas as pd
confine_seccomp = pd.DataFrame(columns=["appNm","seccomp_syscall","eid"])

for loc, appNm in enumerate(confine_profile):
    flat_seccomp = set()
    for syscall in confine_profile[appNm].name:
        flat_seccomp.add(syscall)
    confine_seccomp.loc[loc]=[appNm, flat_seccomp, set()]

confine_seccomp.head()

,appNm,seccomp_syscall,eid
0,celery,"{mlock2, reboot, setdomainname, memfd_create, ...",{}
1,sematext-agent-monitoring-and-logging,"{dup3, ppoll, mlock2, pause, reboot, setdomain...",{}
2,buildpack-deps,"{dup3, ppoll, mlock2, pause, reboot, setdomain...",{}
3,elasticsearch,"{times, mlock2, pause, reboot, setdomainname, ...",{}
4,jenkins,"{dup3, ppoll, mlock2, pause, reboot, setdomain...",{}


In [29]:
# system call name -> num 바꾸기
for index, row in confine_seccomp.iterrows():
    num_seccomp=set()
    for syscall in row.seccomp_syscall:
        sysnum = convert_name_num(syscall)
        num_seccomp.add(sysnum)
    confine_seccomp.loc[index, 'seccomp_syscall'] = num_seccomp

In [39]:
import pickle

# pickle로 저장
with open("confine_seccomp.pkl","wb") as f:
    pickle.dump(confine_seccomp, f)
    
confine_seccomp.head()

,appNm,seccomp_syscall,eid
0,celery,"{27, 159, 223, 154, 187, 29, 286, 246, 179, 27...",{}
1,sematext-agent-monitoring-and-logging,"{27, 159, 223, 154, 187, 29, 286, 246, 261, 17...",{}
2,buildpack-deps,"{27, 159, 223, 154, 187, 29, 286, 246, 261, 17...",{}
3,elasticsearch,"{27, 159, 223, 154, 187, 286, 246, 179, 272, 2...",{}
4,jenkins,"{159, 223, 154, 187, 286, 246, 179, 272, 237, ...",{}


In [44]:
# pickle로 저장
import pickle

with open("confine_seccomp.pkl","rb") as f:
    confine_seccomp = pickle.load(f)

In [49]:
# Non-mitigable Exploit Code 조사

for index, row in confine_seccomp.iterrows():
    eids = set()
    for eid, paths in eid_all_path_dict.items():
        for path in paths:
            left = set(path) - row.seccomp_syscall
            if not left:
                eids.add(eid)
    confine_seccomp.loc[index, 'eid'] = eids
    
confine_seccomp.head()

,appNm,seccomp_syscall,eid
0,celery,"{27, 159, 223, 154, 187, 29, 286, 246, 179, 27...","{47170, 50135, 12, 43345, 40871, 44298, 3, 450..."
1,sematext-agent-monitoring-and-logging,"{27, 159, 223, 268, 154, 187, 29, 286, 246, 26...","{47170, 50135, 1397, 12, 9083, 43345, 40871, 4..."
2,buildpack-deps,"{27, 159, 223, 154, 187, 29, 286, 246, 261, 17...","{47170, 50135, 1397, 12, 9083, 43345, 40871, 4..."
3,elasticsearch,"{27, 159, 223, 154, 187, 286, 246, 179, 272, 2...","{47170, 50135, 12, 9083, 43345, 40871, 44298, ..."
4,jenkins,"{159, 223, 154, 187, 286, 246, 179, 272, 237, ...","{47170, 50135, 12, 9083, 43345, 40871, 44298, ..."


In [50]:
for index, row in confine_seccomp.iterrows():
    print(row.appNm, len(row.eid))

celery 13
sematext-agent-monitoring-and-logging 18
buildpack-deps 17
elasticsearch 16
jenkins 15
softwareag-commandcentral 13
couchdb 13
kapacitor 12
redmine 17
registry 15
fedora 13
ruby 20
adminer 15
xwiki 13
express-gateway 14
django 16
postfixadmin 16
apm-server 13
nuxeo 12
tomee 15
kaazing-gateway 15
lightstreamer 14
ubuntu-upstart 13
mongo 14
alpine 15
mediawiki 16
vault 11
elixir 13
nextcloud 14
odoo 13
zookeeper 15
nats 33
nats-streaming 44
mariadb 13
mysql 15
convertigo 17
rethinkdb 13
orientdb 14
gazebo 11
oraclelinux 14
jetty 15
drupal 16
memcached 16
sonarqube 15
anaconda 19
python 18
ubuntu 17
ghost 15
clearlinux 17
hipache 15
oracle-database-enterprise-edition 14
heartbeat 16
opensuse 17
ibm-security-access-manager 13
softwareag-webmethods-microservicesruntime 15
ibm-db2-warehouse-ee 9
httpd 16
euleros 17
oracle-instant-client 17
geonetwork 15
alt 13
influxdb 17
metricbeat 13
irssi 17
golang 17
julia 14
percona 16
skopos 17
iojs 16
photon 17
perl 17
traefik 17
postgres 13

In [52]:
# 엑셀로 저장하기
confine_seccomp.to_excel("confine_seccomp.xlsx")